In [3]:
%pip install flask flask-cors langchain_google_genai PyMuPDF werkzeug langchain

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ------------------------------- -------- 0.8/1.0 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 4.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ----------------------------- ---------- 1.6/2.1 MB 7.6 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 7.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import json
import re
import fitz  # PyMuPDF
import os
import getpass
from flask import Flask, request, jsonify, render_template
from flask_cors import CORS
from werkzeug.utils import secure_filename
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
# from pyngrok import ngrok, conf  # Ngrok for Colab

In [ ]:
# ✅ Securely get Ngrok Token & Gemini API Key
# conf.get_default().auth_token = getpass.getpass("Enter your ngrok auth token: ")
gemini_api_key = getpass.getpass("Enter your Gemini API Key: ")


Enter your ngrok auth token: ··········
Enter your Gemini API Key: ··········


In [50]:
# ✅ Initialize Flask App
app = Flask(__name__, template_folder="templates")
CORS(app)

In [51]:
# ✅ Define File Upload Directory
UPLOAD_FOLDER = os.path.join(os.getcwd(), "uploads")
os.makedirs(UPLOAD_FOLDER, exist_ok=True)  # Create the folder if it doesn't exist
app.config["UPLOAD_FOLDER"] = UPLOAD_FOLDER  # ✅ Store it in Flask config

In [52]:
# ✅ Load Gemini Model using LangChain
llm = GoogleGenerativeAI(model="gemini-2.0-flash-thinking-exp-01-21", google_api_key=gemini_api_key)

In [53]:
prompt_template = PromptTemplate(
    input_variables=["pitch_text"],
    template="""
    You are an AI assistant specializing in analyzing startup pitch decks.

    **Startup Pitch Deck Content:**
    {pitch_text}

    **Your Task:**
    - Assign an **Overall Pitch Score (0-100)** based on clarity, structure, and business viability.
    - Identify **Strengths** (What is well explained?).
    - Identify **Weaknesses** (What is missing or unclear?).
    - Suggest **Improvements** (How can the pitch be improved?).

    **Return ONLY in this strict JSON format (DO NOT include extra text):**
    ```json
    {{
        "Overall Pitch Score": <score>,
        "Strengths": "<summary>",
        "Weaknesses": "<summary>",
        "Improvement Suggestions": "<summary>"
    }}
    ```
    """
)


In [54]:
# ✅ Extract Text from PDFs
def extract_text_from_pdf(pdf_path):
    """Extracts text from a given PDF file."""
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text.strip()

In [55]:
def extract_json(response_text):
    """Extracts valid JSON from Gemini's response using regex."""
    try:
        return json.loads(response_text)  # ✅ Directly parse JSON if possible
    except json.JSONDecodeError:
        try:
            json_match = re.search(r"\{.*\}", response_text, re.DOTALL)  # ✅ Extract JSON using regex
            if json_match:
                return json.loads(json_match.group())  # ✅ Convert extracted JSON
        except json.JSONDecodeError:
            pass
    return {"error": "Invalid JSON response from Gemini"}  # ✅ Return error if parsing fails

In [56]:

def analyze_pitch(pitch_text):
    """Uses LangChain with Gemini API to analyze the pitch."""
    chain = LLMChain(llm=llm, prompt=prompt_template)
    response = chain.run(pitch_text=pitch_text)  # ✅ Gemini API call

    # ✅ Extract JSON correctly
    return extract_json(str(response))  # Convert to string before parsing


In [57]:
# ✅ Home Route
@app.route("/")
def home():
    return render_template("index.html")

In [58]:
# ✅ Flask Route: Upload & Analyze Pitch Deck
@app.route("/upload", methods=["POST"])
def upload_pdf():
    if "file" not in request.files:
        return jsonify({"error": "No file uploaded"}), 400

    file = request.files["file"]
    if file.filename == "":
        return jsonify({"error": "No selected file"}), 400

    if file:
        filename = secure_filename(file.filename)
        file_path = os.path.join(app.config["UPLOAD_FOLDER"], filename)
        file.save(file_path)

        # ✅ Extract text & analyze
        pitch_text = extract_text_from_pdf(file_path)
        print(pitch_text)
        analysis = analyze_pitch(pitch_text)
        print(analysis)

        return jsonify(analysis)


In [ ]:
# ✅ Run Flask with Ngrok in Colab
if __name__ == "__main__":
    port = 5000
    # public_url = ngrok.connect(port).public_url
    # print(f"🚀 Public URL: {public_url}")
    app.run(port=port)

🚀 Public URL: https://1669-34-82-249-86.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/Mar/2025 12:09:13] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Mar/2025 12:09:14] "GET /favicon.ico HTTP/1.1" 404 -


Pitch Deck Example

GOAL
• The goal of your pitch deck is to give a snapshot of 
your investment opportunity, taking into consideration 
your impact, growth potential, viability, and the ability 
of your team to execute your plan. 
• Think of this as the most compelling elements of your 
executive summary presented visually and verbally.

Housekeeping
• Keep in mind that you will be allotted 10 minutes or 
less to present and may need to adjust the content 
and number of slides accordingly. 
• Ensure slides are not text or data heavy and use at 
least a 30pt font.

TAGLINE: Define your 
venture with a short 
declarative sentence, 
getting to the heart of 
your venture’s unique 
advantage. 
DATE: 
PRESENTER NAME:
Cover page

Problem
“The goal is to get everyone nodding and buying in.” 
- Guy Kawasaki, The Art of the Start
• Describe the pain of the customer (or the customer’s 
customer) 
• Outline how the customer addresses the issue today 
and why current solutions don’t work. 
• Is it

INFO:werkzeug:127.0.0.1 - - [21/Mar/2025 12:09:32] "POST /upload HTTP/1.1" 200 -


{'Overall Pitch Score': 85, 'Strengths': 'The pitch deck template provides a well-structured and comprehensive outline covering all essential sections for a startup pitch. It logically progresses from problem to solution, market, business model, and team. The inclusion of expert quotes and tips throughout the template adds credibility and practical guidance. The emphasis on customer pain, market sizing, competitive advantage, and financial viability is strong. It encourages a focus on key metrics and realistic assumptions.', 'Weaknesses': "While comprehensive, the template is somewhat generic and lacks specific guidance on visual presentation and storytelling. Some sections could benefit from more detailed prompts to ensure deeper thinking (e.g., 'Go-to-Market Strategy' is broad). It doesn't explicitly emphasize the need for a compelling narrative or emotional connection.  The 'Financial Snapshot' could be more specific about key financial metrics beyond revenue and net income. There's

INFO:werkzeug:127.0.0.1 - - [21/Mar/2025 12:10:07] "POST /upload HTTP/1.1" 200 -


{'Overall Pitch Score': 75, 'Strengths': "Clearly articulates the problem and solution, supported by market validation data from Couchsurfing and Craigslist.  The business model is simple and easy to understand (10% commission). The deck effectively highlights key competitive advantages like 'First to Market' and 'Ease of Use'. Market size is quantified, demonstrating a large potential opportunity.", 'Weaknesses': "The 'Product' slide is extremely basic and lacks detail about features and user experience. There are inconsistencies in the 'Business Model' slide regarding revenue calculations (AVG FEE and REVENUE figures are misaligned). The 'Competitive Advantages' slide contains unprofessional 'Lorem Ipsum' filler text instead of detailed explanations. Market size definitions (TAM, SAM, Market Share) could be clearer, and the 'Market Share' calculation label 'TRIPS W/AB&B' is confusing.  'Market Adoption' strategies are listed but lack depth and specific execution plans.", 'Improvement